## Data Cleaning Notebook
This notebook contains the relevant code for aggregating our CSV's into a singular one that can then be used in our models. 

Contains the One-Hot encoding, and featurization of the columns. 

In [3]:
import os
import pandas as pd 
import numpy as np 

#Descriptions of sets available at https://www.imdb.com/interfaces/

In [28]:
# Loading the Sets: 
sets = {} 

for file in os.listdir('DataSets'):
    file = file.replace('.tsv', '') 
    sets[file] = pd.read_csv(f"DataSets/{file}.tsv", sep='\t')
    if 'tconst' in sets[file].keys():
        sets[file].set_index('tconst',inplace=True)
    elif 'titleId' in sets[file].keys():
        sets[file].set_index('titleId', inplace=True) 
    else:
        print("Cant set index for ", file)
    print(f"Loaded {file}")
print("Loaded all datasets") 

Loaded title.ratings
Loaded title.principals
Loaded title.akas
Cant set index for  name.basics
Loaded name.basics
Loaded title.basics
Loaded title.crew
Loaded all datasets


# What we need from each dataset: 
### title.akas 
    * Language
    * region 
### title.basics
    * titleType -> Used to only keep movies, used in filtering. We should filter before we split, etc. 
    * primaryTitle -> We only care about the most popular title so use this one 
    * isAdult -> Why not include these 
    * startYear -> Year movie released, good to see to potentially capture trends of a timeperiod 
    * runtimeMinutes -> Good to know for how long a movie is 
    * genres -> One hot encoding on the genres it has 
### title.crew 
    * directors -> One hot encoding on top 100 directors 
    * writers -> One hot encoding on top 100 writers 
### title.episode 
    * We dont care about episodes so skip this one 
### title.principals  - Info about cast / crew for titles 
    * nconst -> Useful for determining which actor is who. Can use this with one-hot encoding for each movie to 
                determine top 100 actors and whether or not they were in a movie or not 
### title.ratings 
    * averageRating -> Weight average of all individual ratings, used as our target variable 
    * numVotes -> The number of votes it received - useful to somehow include this in our target, would want to 
                  weight training samples with move votes w/ more importance, can be used with models that allow that 
### name.basics
    * Can potentially include this later on in our featurizations if we need info about the people, currently I think 
      just their unique id from title.principals should be more than enough to capture actors. 

# Data Cleaning: 
    1) First get ids of all entries that are movies (So we are not including things that arent movies) 
    
    2) Then filter the raitings CSV to only include just movies, then move to actually just only keep the first X number of movies with a certain raitings count. 

In [46]:
print("DFs we have loaded: ", sets.keys())

DFs we have loaded:  dict_keys(['title.ratings', 'title.principals', 'title.akas', 'name.basics', 'title.basics', 'title.crew'])


In [47]:
## FILTERING SO WE ONLY HAVE MOVIES: 
df = sets['title.basics']

# Keep only the rows that have a titleType of movie
df = df[df['titleType'] == 'movie']

# The row ids that are just movies 
movie_ids = list(df.index)
#first 10 just to make sure its ids 
print(movie_ids[:10])


['tt0000502', 'tt0000574', 'tt0000591', 'tt0000615', 'tt0000630', 'tt0000675', 'tt0000679', 'tt0000739', 'tt0000793', 'tt0000814']


In [49]:
# Shifting over to the title.ratings CSV to get the first X number of movies with the most reviews 
df = sets['title.ratings']

#Filtering the df to only include the ids that were explicitly movies 
df = df.loc[movie_ids]

# Sorting the df by numVotes so its from most number of reviews -> less 
df.sort_values(by='numVotes', inplace=True, ascending=False)
# I dont know why I hardcoded these but checks 100k to 20k in batches of 10k  
for splitter in [100000, 90000, 80000, 70000, 60000, 50000, 40000, 30000, 20000]:
    top = list(df['numVotes'][:splitter])
    print(f"Total number of movies: {splitter} : Lowest raiting count: {top[-1]}")


# We will go with 30k for now to keep it simple :) 
TOTAL_MOVIES_TO_KEEP = 30000 

Total number of movies: 100000 : Lowest raiting count: 130.0
Total number of movies: 90000 : Lowest raiting count: 166.0
Total number of movies: 80000 : Lowest raiting count: 216.0
Total number of movies: 70000 : Lowest raiting count: 288.0
Total number of movies: 60000 : Lowest raiting count: 395.0
Total number of movies: 50000 : Lowest raiting count: 569.0
Total number of movies: 40000 : Lowest raiting count: 870.0
Total number of movies: 30000 : Lowest raiting count: 1481.0
Total number of movies: 20000 : Lowest raiting count: 2997.0


/Users/ryankennedy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """


In [50]:
# The ids that meet the criteria we are utilizing now in our main df. 
movie_ids = list(df['numVotes'][:TOTAL_MOVIES_TO_KEEP].index)
# Filter the DataFrame to only keep those ids :) 
print(df.shape)
df = df.loc[movie_ids]
print(df.shape)

# Our output final df, named final_df for ease. Contains the top 30k movies for the number of votes. 
final_df = df 

(606395, 2)
(30000, 2)


In [64]:
# If we ever need to change the columns we are merging into it, do so here :) 
AKA_COLS = ['region', 'language'] 
BASICS_COLS = ['primaryTitle', 'isAdult', 'startYear', 'runtimeMinutes', 'genres']

# Grab the respective dfs, only grab the rows that are our movie ids we have chosen to work with 
df_aka = sets['title.akas'].loc[movie_ids]
df_basic = sets['title.basics'].loc[movie_ids]


/Users/ryankennedy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  


In [63]:
# Filtering the dfs to only be the columns we are actually interested in 
df_aka = df_aka[AKA_COLS]
df_basic = df_basic[BASICS_COLS]

# Merging AKA and Basic 
merged = df_aka.join(df_basic)

# Joining the merged above df into our final df 
final_df = final_df.join(merged) 
final_df

,averageRating,numVotes,region,language,primaryTitle,isAdult,startYear,runtimeMinutes,genres
tt0002130,7.0,2815.0,FI,\N,Dante's Inferno,0,1911,71,"Adventure,Drama,Fantasy"
tt0002130,7.0,2815.0,HU,\N,Dante's Inferno,0,1911,71,"Adventure,Drama,Fantasy"
tt0002130,7.0,2815.0,IT,\N,Dante's Inferno,0,1911,71,"Adventure,Drama,Fantasy"
tt0002130,7.0,2815.0,\N,\N,Dante's Inferno,0,1911,71,"Adventure,Drama,Fantasy"
tt0002130,7.0,2815.0,US,\N,Dante's Inferno,0,1911,71,"Adventure,Drama,Fantasy"
...,...,...,...,...,...,...,...,...,...
tt9916362,6.4,4523.0,PL,\N,Coven,0,2020,92,"Drama,History"
tt9916362,6.4,4523.0,AR,\N,Coven,0,2020,92,"Drama,History"
tt9916362,6.4,4523.0,XWW,en,Coven,0,2020,92,"Drama,History"
tt9916362,6.4,4523.0,PT,\N,Coven,0,2020,92,"Drama,History"


In [66]:
CREW_COLS = ['directors','writers']
# Merging The Crew df into our main Df: Should probably do one hot encoding here before we actually merge it but idk 
crew = sets['title.crew'].loc[movie_ids]
crew = crew[CREW_COLS]

In [67]:
print(crew) 

            directors                                            writers
tconst                                                                  
tt0111161   nm0001104                                nm0000175,nm0001104
tt0468569   nm0634240            nm0634300,nm0634240,nm0275286,nm0004170
tt1375666   nm0634240                                          nm0634240
tt0137523   nm0000399                                nm0657333,nm0880243
tt0109830   nm0000709                                nm0343165,nm0744839
...               ...                                                ...
tt1674775   nm0534665                                nm0866014,nm0534665
tt5795282   nm0231999                                                 \N
tt0027489   nm0656528                                          nm0656528
tt13074322  nm8585595                                          nm8585595
tt0109098   nm0203246  nm0189485,nm0203246,nm0295165,nm0668898,nm0672654

[30000 rows x 2 columns]


In [82]:
# Grabs the top X number of categories, defaulted to 100. Utilized by our one-hot encoding system to
# determine what the most used number of categories are 
def top_categories(df, col_name, TOTAL_TO_KEEP = 100):
    """
        Provided a dataframe and the column name, returns the top 100 categories for this column 
        @param df: The DataFrame we are searching over 
        @param col_name: The column name we are examining the categories for 
        @param TOTAL_TO_KEEP: Constant specifying the total number of entries to keep. 
        :return A list of the top 100 categories for this column. 
    """
    frequencies = {}
    for entry in list(df[col_name]):
        # If we have a list of items iterate over each of them 
        if not isinstance(entry, str) and isinstance(entry, list):
            for entry_sub in entry:
                if entry_sub in frequencies:
                    frequencies[entry_sub] += 1
                else:
                    frequencies[entry_sub] = 1 
        else:
            # Just a singular item so can compare it here directly. 
            if entry in frequencies:
                frequencies[entry] += 1
            else:
                frequencies[entry] = 1 
    
    lof_frequencies = []
    for key in frequencies.keys():\
        lof_frequencies.append( (key, frequencies[key]) )
    lof_frequencies.sort(key = (lambda pair: pair[1]))
        
    return [pair[0] for pair in lof_frequencies[:TOTAL_TO_KEEP]]


def encode_row(row, category):
    """
        Encodes a specific row / entry in our dataframe. If the row is a list, checks to see if the 
        category value exists in it, if it is not a list just checks to see if the row is equal to the category 
    """
    if not isinstance(row, str) and isinstance(row, list):
        return 1 if category in row else 0 
    else:
        return 1 if category == row else 0 
    

# Converts a specific column to a one-hot encoding version of it.
def encode_column(df, col_name, TOTAL_TO_KEEP = 100):
    """
        Provided a dataframe and column to encode, mutates the dataframe to have that a one-hot encoding of that 
        specific column. Will remove that specific column from the dataframe and replace it with TOTAL_TO_KEEP 
        columns for that value plus one more column to handle any categorical variable that was not in the top 
        TOTAL_TO_KEEP categories. 
        
        @param df: The dataframe we are mutating 
        @param col_name: The column we are encoding, this column is removed from the df and replaced with the encodings
        @param TOTAL_TO_KEEP: Number of categories we want to display, defaulted to 100 
    """
    categories = top_categories(df, col_name, TOTAL_TO_KEEP=TOTAL_TO_KEEP)
    
    # Our encoded columns for the dataset goes here 
    encoded_cols = {}
    
    lof_column = list(df[col_name])
    
    for category in categories: 
        encoded_column = [encode_row(row, category) for row in lof_column]
        
        encoded_col_name = f"{col_name}_{category}"
        df[encoded_col_name] = encoded_column
    
    


,directors,writers,"directors_nm0591450,nm0021249","directors_nm0881279,nm0004056","directors_nm0677037,nm0230032",directors_nm0005124,"directors_nm0230032,nm0798899,nm0881279","directors_nm0849164,nm0000965","directors_nm0751648,nm0751577",directors_nm0881279,...,"directors_nm1224299,nm0970447",directors_nm0851281,directors_nm0727754,"directors_nm0971239,nm9054338",directors_nm2193504,"directors_nm0527217,nm0314671,nm0414144",directors_nm1844237,"directors_nm0601781,nm1601882","directors_nm0201509,nm0426333","directors_nm0796923,nm0000142"
tconst,,,,,,,,,,,,,,,,,,,,,
tt0111161,nm0001104,"nm0000175,nm0001104",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0468569,nm0634240,"nm0634300,nm0634240,nm0275286,nm0004170",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt1375666,nm0634240,nm0634240,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0137523,nm0000399,"nm0657333,nm0880243",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0109830,nm0000709,"nm0343165,nm0744839",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt1674775,nm0534665,"nm0866014,nm0534665",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt5795282,nm0231999,\N,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0027489,nm0656528,nm0656528,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
